In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pacswg

In [2]:
# TODO: measurements are once every 30 seconds, convert them to time

In [3]:
# service_name = 'bentoml-keras-toxic-comments'
# exp_name = 'res-2021-06-28_22-09-08'
# slo_timeout = 500
# trace_name = 'trace2'

service_name = 'bentoml-pytorch-fashion-mnist'
exp_name = 'res-2021-07-28_18-06-59'
slo_timeout = 500
trace_name = 'trace_trace_wc'

exp_folder = f'results/{trace_name}/{service_name}'
exp_proxy_filename = f'{exp_folder}/{exp_name}_proxy.csv'
exp_reqs_filename = f'{exp_folder}/{exp_name}_reqs.csv'

In [4]:
df_proxy_stats = pd.read_csv(exp_proxy_filename, index_col=0, parse_dates=True)
df_res = pd.read_csv(exp_reqs_filename, index_col=0, parse_dates=True)

FileNotFoundError: [Errno 2] No such file or directory: 'results/trace_trace_wc/res-2021-07-28_18-06-59/res-2021-06-28_22-09-08_proxy.csv'

In [ ]:
plt.plot(df_proxy_stats['averageMaxBufferSize'])
plt.plot(df_proxy_stats['averageActualBatchSize'])

In [ ]:
plt.plot(df_proxy_stats['averageTimeoutRatio'])

In [ ]:
plt.plot(df_proxy_stats['reponseTimeAverage'])
plt.plot(df_proxy_stats['reponseTimeP50'])
plt.plot(df_proxy_stats['reponseTimeP95'])

In [ ]:
plt.plot(df_proxy_stats['reponseTimeAverage'][5:30])
plt.plot(df_proxy_stats['reponseTimeP50'][5:30])
plt.plot(df_proxy_stats['reponseTimeP95'][5:30])

In [ ]:
plt.plot(df_proxy_stats['currentReplicaCount'])
plt.plot(df_proxy_stats['currentReadyReplicaCount'], ls='--')
print(df_proxy_stats['currentReadyReplicaCount'].mean())

In [ ]:
plt.plot(df_proxy_stats['averageArrivalRate'])

## Other Useful Plots

In [ ]:
df_res['received_at'] = pd.to_datetime(df_res['received_at'])
df_res['response_at'] = pd.to_datetime(df_res['response_at'])

In [ ]:
# get a temporal data of means
temp_df = df_res.set_index('received_at').resample('T').mean()
temp_df.head()

In [ ]:

plt.plot(temp_df['upstream_response_time'])

In [ ]:
plt.plot(temp_df['upstream_request_count'])

In [ ]:
plt.plot(temp_df['queue_time_ms'], label='Queue Time (ms)')
plt.plot(temp_df['response_time_ms_server'], label='Response Time (ms)')
plt.plot(temp_df['upstream_response_time'], label='Upstream RT (ms)')

plt.legend()

In [ ]:
df_resample = df_res.set_index('received_at').resample('T')
p95_over_time = df_resample['response_time_ms_server'].quantile(0.95)
plt.plot(p95_over_time)

In [ ]:
# SLO miss rate
slo_miss_count = df_resample['response_time_ms_server'].apply(lambda x: np.sum(x > slo_timeout))
resampled_request_count = df_resample['response_time_ms_server'].apply(lambda x: len(x))
slo_miss_rates = slo_miss_count / resampled_request_count * 100
plt.plot(slo_miss_rates)
plt.ylabel('SLO Miss Rate (%)')
plt.xlabel('Time')